# 과제 1 네이버 평점 데이터 수집

In [1]:
import pandas as pd
ratings = pd.read_csv('./movie-recsys-namsanghoon-master/data/naver_user.csv')
ratings

reviewNo userId
0   15772038   airf
1   15772037   nanw
2   15772036   zxcv
3   15772035   sdh1
4   15772032   guan
5   15772031   kalk
6   15772030   xmj0
7   15772029   tmfr
8   15772028   glab
9   15772027   thdu
10  15772026   0424
11  15772025   yoon
12  15772023   dudd
13  15772022   k2sn
14  15772021   kin0
15  15772020   zct2
16  15772019   wjda
17  15772018   lily
18  15772017   huyr
19  15772016   nege
20  15772015   frie
21  15772014   vanq
22  15772013   youn
23  15772012   zard
24  15772011   hane
25  15772010   lse0
26  15772009   scw6
27  15772008   mac2
28  15772007   h471
29  15772006   taix
..       ...    ...
70  15771958   kimd
71  15771956   msjs
72  15771955   qudr
73  15771954   gopa
74  15771953   yyua
75  15771951   babo
76  15771950   bjwb
77  15771948   artn
78  15771947   sugo
79  15771946   sohe
80  15771945   grea
81  15771944   zero
82  15771943   toyc
83  15771942   lucy
84  15771941   hall
85  15771940   imag
86  15771939   jump
87  15771938   bs07
88  15771937   reds
89  15771936   ykm3
90  15771935   jjo2
91  15771934   tsp0
92  15771933   sky9
93  15771932   yodo
94  15771931   dewp
95  15771929   eyku
96  15771927   geon
97  15771926   myil
98  15771925   hous
99  15771922   huya

[100 rows x 2 columns]

In [2]:
rdf = pd.DataFrame(columns=("userId", "rating", "movieId"))
rdf

Empty DataFrame
Columns: [userId, rating, movieId]
Index: []

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import re
import random
import time
import pandas as pd
import requests 


driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(3)

def get_user_list(limit):
    page = 1
    user_list = []
    while len(user_list) <= limit:
        driver.get('https://movie.naver.com/movie/point/af/list.nhn?&page={0}'.format(page))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        reviewNum = soup.select('#old_content > table > tbody > tr > td.ac.num')
        userId = soup.select('table.list_netizen > tbody > tr > td > a.author')

        for i in range(len(reviewNum)):
            review = str(reviewNum[i]).replace('<td class="ac num">','').replace('</td>','')
            user = userId[i].text.replace('*','')
            if user not in [user[1] for user in user_list]:
                user_list.append((review, user))

        time.sleep(random.randrange(2, 5))
        page += 1
    
    return user_list[:limit]

def Deduplication(reviewNo,page):
    temp=[]
    diction=[]
    driver.get('https://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword={0}&target=after/&page={1}'.format(reviewNo,page))
    html =driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    movie_links = get_movie_link(soup)
    movieId = [link.replace('https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=','').replace('&target=after','') for link in movie_links]
    
    a = soup.find("table", {"class": "list_netizen"})
    tbody = a.find('tbody')
    trs = tbody.find_all('tr')
    for tr in trs:
        tds = tr.find_all('td',{"class":"point"})
        for td in tds:
            temp.append(td.text)
    
    for i in range(len(movieId)):
        dic = {temp[i] : movieId[i]}
        diction.append(dic)           
               
    return diction
    
    

def get_movie_link(soup):
    movie_links = soup.select('a[href]')

    movie_links_list = []
    for link in movie_links:
        if re.search(r'st=mcode&sword' and r'&target=after$', link['href']):
            target_url = 'https://movie.naver.com/movie/point/af/list.nhn'+str(link['href'])
            movie_links_list.append(target_url)
    
    return movie_links_list[1:]

def get_review(reviewNo):
    user_reviews = []
    rating=[]
    title=[]
    
    page=1
    while True:
        driver.get('https://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword={0}&target=after/&page={1}'.format(reviewNo,page))
        html =driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        htnm_migration_table = soup.find("table", {"class": "list_netizen"})
        tbody = htnm_migration_table.find('tbody')
        trs = tbody.find_all('tr')
        
        for tr in trs:
            tda = tr.find('td',{"class":"ac num"})
            
            review = str(tda).replace('<td class="ac num">','').replace('</td>','')
            if int(review) <= reviewNo:
                tds = tr.find_all('td',{"class":"point"})
                for td in tds:
                    rating.append(td.text)
                title_td = tr.find_all('a',{"class":"movie"})
                for td in title_td:
                    movieid = str(td).replace('<a class="movie" href="?st=mcode&amp;sword=','')
                    title.append(movieid.split('&')[0])    
        
        if Deduplication(reviewNo,page) == Deduplication(reviewNo,page+1):
            break
        
        else:
            page +=1
    time.sleep(random.randrange(3, 5))
    for i in range(len(rating)):
            dic = {rating[i] : title[i]}
            user_reviews.append(dic)          
    return user_reviews

def main():
    count=0
    for j in range(len(ratings)):
            rating = get_review(ratings['reviewNo'][j])
            for i in range(len(rating)):
                rdf.loc[count] = {'userId' : ratings['userId'][j],'rating':list(rating[i].keys())[0],'movieId':list(rating[i].values())[0]}
                count = count + 1
    
    
    

if __name__ == "__main__":
    main()

In [4]:
rdf

userId rating movieId
0      airf      2  136900
1      airf     10  163788
2      airf     10  174065
3      nanw     10  154667
4      nanw     10  136900
5      nanw     10  145335
6      nanw      9  120141
7      nanw      9   54704
8      zxcv      7  113351
9      zxcv     10   86507
10     zxcv     10   39705
11     zxcv      1   39756
12     zxcv     10   31925
13     zxcv      6   35098
14     zxcv      1  168679
15     zxcv      8  123300
16     zxcv      2  151188
17     zxcv      2   41592
18     zxcv     10   18811
19     zxcv      1  118994
20     zxcv      2  164183
21     zxcv     10   76016
22     zxcv     10  130997
23     zxcv     10   24522
24     zxcv      1  148909
25     zxcv      2  139700
26     zxcv      1  158622
27     zxcv     10  171725
28     zxcv      6  181411
29     zxcv      7  164932
...     ...    ...     ...
5088   hous      5  118950
5089   hous      6   77768
5090   hous      7   98438
5091   hous      1   93218
5092   hous     10  109642
5093   hous     10  129266
5094   hous      7   45290
5095   hous     10   65113
5096   hous     10   91192
5097   hous      3   71073
5098   hous     10   17421
5099   huya     10  152331
5100   huya      6  153580
5101   huya      1  157297
5102   huya      1   96917
5103   huya     10   66215
5104   huya      9   16822
5105   huya     10  121051
5106   huya      1  112079
5107   huya      4  103759
5108   huya      1   99800
5109   huya      9   35939
5110   huya      9   68616
5111   huya     10   45889
5112   huya      9   49637
5113   huya     10   73394
5114   huya     10   70773
5115   huya      8   16220
5116   huya     10   36666
5117   huya     10   37235

[5118 rows x 3 columns]

In [5]:
rdf.loc[5118] = ['patl',10,'161967']
rdf.loc[5119] = ['yoya',10,'161967']
rdf.loc[5120] = ['ebc8',1,'174065']
rdf.loc[5121] = ['imag',10,'161967']
rdf.loc[5122] = ['glab',8,'137327']
rdf.loc[5123] = ['zxcv',1,'174065']
rdf.loc[5124] = ['kktw',7,'47701']
rdf.loc[5125] = ['kktw',6,'62328']
rdf

userId rating movieId
0      airf      2  136900
1      airf     10  163788
2      airf     10  174065
3      nanw     10  154667
4      nanw     10  136900
5      nanw     10  145335
6      nanw      9  120141
7      nanw      9   54704
8      zxcv      7  113351
9      zxcv     10   86507
10     zxcv     10   39705
11     zxcv      1   39756
12     zxcv     10   31925
13     zxcv      6   35098
14     zxcv      1  168679
15     zxcv      8  123300
16     zxcv      2  151188
17     zxcv      2   41592
18     zxcv     10   18811
19     zxcv      1  118994
20     zxcv      2  164183
21     zxcv     10   76016
22     zxcv     10  130997
23     zxcv     10   24522
24     zxcv      1  148909
25     zxcv      2  139700
26     zxcv      1  158622
27     zxcv     10  171725
28     zxcv      6  181411
29     zxcv      7  164932
...     ...    ...     ...
5096   hous     10   91192
5097   hous      3   71073
5098   hous     10   17421
5099   huya     10  152331
5100   huya      6  153580
5101   huya      1  157297
5102   huya      1   96917
5103   huya     10   66215
5104   huya      9   16822
5105   huya     10  121051
5106   huya      1  112079
5107   huya      4  103759
5108   huya      1   99800
5109   huya      9   35939
5110   huya      9   68616
5111   huya     10   45889
5112   huya      9   49637
5113   huya     10   73394
5114   huya     10   70773
5115   huya      8   16220
5116   huya     10   36666
5117   huya     10   37235
5118   patl     10  161967
5119   yoya     10  161967
5120   ebc8      1  174065
5121   imag     10  161967
5122   glab      8  137327
5123   zxcv      1  174065
5124   kktw      7   47701
5125   kktw      6   62328

[5126 rows x 3 columns]

In [6]:
rdf=rdf.drop_duplicates().reset_index()
rdf['movieId'] = rdf['movieId'].astype("int")
rdf

index userId rating  movieId
0         0   airf      2   136900
1         1   airf     10   163788
2         2   airf     10   174065
3         3   nanw     10   154667
4         4   nanw     10   136900
5         5   nanw     10   145335
6         6   nanw      9   120141
7         7   nanw      9    54704
8         8   zxcv      7   113351
9         9   zxcv     10    86507
10       10   zxcv     10    39705
11       11   zxcv      1    39756
12       12   zxcv     10    31925
13       13   zxcv      6    35098
14       14   zxcv      1   168679
15       15   zxcv      8   123300
16       16   zxcv      2   151188
17       17   zxcv      2    41592
18       18   zxcv     10    18811
19       19   zxcv      1   118994
20       20   zxcv      2   164183
21       21   zxcv     10    76016
22       22   zxcv     10   130997
23       23   zxcv     10    24522
24       24   zxcv      1   148909
25       25   zxcv      2   139700
26       26   zxcv      1   158622
27       27   zxcv     10   171725
28       28   zxcv      6   181411
29       29   zxcv      7   164932
...     ...    ...    ...      ...
5094   5096   hous     10    91192
5095   5097   hous      3    71073
5096   5098   hous     10    17421
5097   5099   huya     10   152331
5098   5100   huya      6   153580
5099   5101   huya      1   157297
5100   5102   huya      1    96917
5101   5103   huya     10    66215
5102   5104   huya      9    16822
5103   5105   huya     10   121051
5104   5106   huya      1   112079
5105   5107   huya      4   103759
5106   5108   huya      1    99800
5107   5109   huya      9    35939
5108   5110   huya      9    68616
5109   5111   huya     10    45889
5110   5112   huya      9    49637
5111   5113   huya     10    73394
5112   5114   huya     10    70773
5113   5115   huya      8    16220
5114   5116   huya     10    36666
5115   5117   huya     10    37235
5116   5118   patl     10   161967
5117   5119   yoya     10   161967
5118   5120   ebc8      1   174065
5119   5121   imag     10   161967
5120   5122   glab      8   137327
5121   5123   zxcv      1   174065
5122   5124   kktw      7    47701
5123   5125   kktw      6    62328

[5124 rows x 4 columns]

In [7]:
user_num = rdf.userId.unique().shape[0]
movie_num = rdf.movieId.unique().shape[0]
print("유저의 수는 {}명 이고, \n영화의 수는 {}개 입니다.".format(user_num,movie_num))

유저의 수는 100명 이고, 
영화의 수는 2697개 입니다.


In [8]:
fdf=pd.merge(rdf,ratings,how='outer')
fdf['movieId'] = fdf['movieId'].astype("int")
fdf['reviewNo'] = fdf['reviewNo'].astype("int")
fdf['rating'] = fdf['rating'].astype("int")
fdf

index userId  rating  movieId  reviewNo
0         0   airf       2   136900  15772038
1         1   airf      10   163788  15772038
2         2   airf      10   174065  15772038
3         3   nanw      10   154667  15772037
4         4   nanw      10   136900  15772037
5         5   nanw      10   145335  15772037
6         6   nanw       9   120141  15772037
7         7   nanw       9    54704  15772037
8         8   zxcv       7   113351  15772036
9         9   zxcv      10    86507  15772036
10       10   zxcv      10    39705  15772036
11       11   zxcv       1    39756  15772036
12       12   zxcv      10    31925  15772036
13       13   zxcv       6    35098  15772036
14       14   zxcv       1   168679  15772036
15       15   zxcv       8   123300  15772036
16       16   zxcv       2   151188  15772036
17       17   zxcv       2    41592  15772036
18       18   zxcv      10    18811  15772036
19       19   zxcv       1   118994  15772036
20       20   zxcv       2   164183  15772036
21       21   zxcv      10    76016  15772036
22       22   zxcv      10   130997  15772036
23       23   zxcv      10    24522  15772036
24       24   zxcv       1   148909  15772036
25       25   zxcv       2   139700  15772036
26       26   zxcv       1   158622  15772036
27       27   zxcv      10   171725  15772036
28       28   zxcv       6   181411  15772036
29       29   zxcv       7   164932  15772036
...     ...    ...     ...      ...       ...
5094   5091   hous       1    93218  15771925
5095   5092   hous      10   109642  15771925
5096   5093   hous      10   129266  15771925
5097   5094   hous       7    45290  15771925
5098   5095   hous      10    65113  15771925
5099   5096   hous      10    91192  15771925
5100   5097   hous       3    71073  15771925
5101   5098   hous      10    17421  15771925
5102   5099   huya      10   152331  15771922
5103   5100   huya       6   153580  15771922
5104   5101   huya       1   157297  15771922
5105   5102   huya       1    96917  15771922
5106   5103   huya      10    66215  15771922
5107   5104   huya       9    16822  15771922
5108   5105   huya      10   121051  15771922
5109   5106   huya       1   112079  15771922
5110   5107   huya       4   103759  15771922
5111   5108   huya       1    99800  15771922
5112   5109   huya       9    35939  15771922
5113   5110   huya       9    68616  15771922
5114   5111   huya      10    45889  15771922
5115   5112   huya       9    49637  15771922
5116   5113   huya      10    73394  15771922
5117   5114   huya      10    70773  15771922
5118   5115   huya       8    16220  15771922
5119   5116   huya      10    36666  15771922
5120   5117   huya      10    37235  15771922
5121   5118   patl      10   161967  15771989
5122   5119   yoya      10   161967  15771960
5123   5120   ebc8       1   174065  15771979

[5124 rows x 5 columns]

In [9]:
top = fdf.groupby(['userId','reviewNo']).size().nlargest(10).reset_index()
username=list()
for i in range(len(top)):
    username.append(top['userId'][i])
top = pd.merge(top,fdf,how='inner')
top

userId  reviewNo    0  index  rating  movieId
0      ykm3  15771936  700   3409       1   137938
1      ykm3  15771936  700   3410       3   145162
2      ykm3  15771936  700   3411       8    86343
3      ykm3  15771936  700   3412       8   127459
4      ykm3  15771936  700   3413       8   127767
5      ykm3  15771936  700   3414      10   153729
6      ykm3  15771936  700   3415      10    98438
7      ykm3  15771936  700   3416       8   152680
8      ykm3  15771936  700   3417       9   125466
9      ykm3  15771936  700   3418       3   127382
10     ykm3  15771936  700   3419       3   158178
11     ykm3  15771936  700   3420      10    45290
12     ykm3  15771936  700   3421       8    12056
13     ykm3  15771936  700   3422      10    87309
14     ykm3  15771936  700   3423       3   143932
15     ykm3  15771936  700   3424       3   157795
16     ykm3  15771936  700   3425       3   163842
17     ykm3  15771936  700   3426       2   152249
18     ykm3  15771936  700   3427       1   148996
19     ykm3  15771936  700   3428       8   149757
20     ykm3  15771936  700   3429       5   154272
21     ykm3  15771936  700   3430       8   156912
22     ykm3  15771936  700   3431       6   163386
23     ykm3  15771936  700   3432       6   161242
24     ykm3  15771936  700   3433       3   137970
25     ykm3  15771936  700   3434       6   127374
26     ykm3  15771936  700   3435       5   151132
27     ykm3  15771936  700   3436       2   123630
28     ykm3  15771936  700   3437      10   150376
29     ykm3  15771936  700   3438       9   125459
...     ...       ...  ...    ...     ...      ...
3742   imag  15771940  102   3272      10   129408
3743   imag  15771940  102   3273      10    38357
3744   imag  15771940  102   3274      10   129406
3745   imag  15771940  102   3275      10    81890
3746   imag  15771940  102   3276      10    10119
3747   imag  15771940  102   3277      10   118323
3748   imag  15771940  102   3278      10   133402
3749   imag  15771940  102   3279       7   121048
3750   imag  15771940  102   3280       5   130787
3751   imag  15771940  102   3281       9    93218
3752   imag  15771940  102   3282       1   136832
3753   imag  15771940  102   3283      10   134133
3754   imag  15771940  102   3284       9   130346
3755   imag  15771940  102   3285       6    87276
3756   imag  15771940  102   3286       9    92536
3757   imag  15771940  102   3287       9   104009
3758   imag  15771940  102   3288       9    61706
3759   imag  15771940  102   3289      10    72362
3760   imag  15771940  102   3290      10    94897
3761   imag  15771940  102   3291       8   122120
3762   imag  15771940  102   3292      10   136034
3763   imag  15771940  102   3293       1   120123
3764   imag  15771940  102   3294      10   114293
3765   imag  15771940  102   3295      10   113348
3766   imag  15771940  102   3296      10   117789
3767   imag  15771940  102   3297       8   118361
3768   imag  15771940  102   3298      10    37951
3769   imag  15771940  102   3299       8    45715
3770   imag  15771940  102   3300       4   100015
3771   imag  15771940  102   5121      10   161967

[3772 rows x 6 columns]

# 2-a 유사 사용자 탐색

In [10]:
top_user = pd.DataFrame(columns=("userId", "rating", "movieId","reviewNo"))
top_user

Empty DataFrame
Columns: [userId, rating, movieId, reviewNo]
Index: []

In [11]:
count=0
for i in range(10):
    top_user.loc[count] = {'userId' : top[top['userId']==username[i]].iloc[1]['userId'],'rating':top[top['userId']==username[i]].iloc[1]['rating'],'movieId':top[top['userId']==username[i]].iloc[1]['movieId'],'reviewNo':top[top['userId']==username[i]].iloc[1]['reviewNo']}
    count+=1
top_user = top_user.head(10)
top_user

userId rating movieId  reviewNo
0   ykm3      3  145162  15771936
1   sang     10  161967  15771961
2   tsp0      7  163788  15771934
3   hosu      7  180399  15771998
4   zxcv     10   86507  15772036
5   zard     10  158653  15772012
6   artn     10  172174  15771948
7   suha      7  180399  15771976
8   ldsl      1  157297  15771977
9   imag      9   52429  15771940

In [12]:
UM_matrix_ds = fdf.pivot(index='reviewNo', columns='movieId',values='rating')
UM_matrix_ds.head(100)


movieId   10002   10003   10004   10005   10006   10008   10009   10012   \
reviewNo                                                                   
15771922     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771925     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771926     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771927     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771929     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771931     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771932     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771933     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771934     NaN     NaN     NaN     NaN     NaN     NaN     NaN     9.0   
15771935     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771936     NaN     NaN     NaN     NaN    10.0     NaN     NaN     NaN   
15771937     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771938     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771939     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771940     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771941     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771942     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771943     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771944     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771945     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771946     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771947     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771948     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771950     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771951     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771953     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771954     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771955     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771956     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15771958     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
...          ...     ...     ...     ...     ...     ...     ...     ...   
15772006     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772007     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772008     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772009     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772010     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772011     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772012     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772013     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772014     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772015     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772016     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772017     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772018     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772019     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772020     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772021     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772022     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772023     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772025     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
15772026     NaN     NaN     NaN     NaN     NaN

In [13]:
def distance_cosine(a, b):
    return 1-distance.cosine(a, b)

In [14]:
def nearest_neighbor_user(user, topN, simFunc):
    ul = UM_matrix_ds.loc[user].dropna()
    ratedIndex = ul.index
    nn= {}
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 =[]
        interSectionU2 =[]
        if uid == user: continue
        
        for i in ratedIndex:
            if not math.isnan(row[i]):
                
                interSectionU1.append(ul[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim): nn[uid] = round(sim,2)
         
    return sorted(nn.items(), key = itemgetter(1))[:-(topN+1):-1]

In [15]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
for user in top_user['reviewNo']:
    print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(user, 3, distance_cosine)))

User 15771936 neighbors : [(15771972, 1.0), (15771993, 0.97), (15772003, 0.95)]
User 15771961 neighbors : [(15771970, 1.0), (15771966, 1.0), (15771947, 1.0)]
User 15771934 neighbors : [(15772009, 1.0), (15771942, 1.0), (15772030, 0.99)]
User 15771998 neighbors : [(15771970, 1.0), (15771965, 1.0), (15771947, 1.0)]
User 15772036 neighbors : [(15771959, 1.0), (15772000, 0.98), (15771966, 0.98)]
User 15772012 neighbors : [(15772003, 1.0), (15772016, 0.95), (15772000, 0.92)]
User 15771948 neighbors : [(15771966, 1.0), (15771945, 1.0), (15771943, 0.98)]
User 15771976 neighbors : [(15771974, 1.0), (15771966, 1.0), (15771947, 1.0)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 1.0), (15771983, 0.99)]
User 15771940 neighbors : [(15771993, 1.0), (15771981, 1.0), (15771947, 1.0)]


In [16]:
def distance_correlation(a, b):
    return 1-distance.correlation(a, b)

In [17]:
for user in top_user['reviewNo']:
    print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(user, 3, distance_correlation)))
  

C:\Users\rrsg  aa   n odd12\Anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


User 15771936 neighbors : [(15771993, 1.0), (15772022, 0.69), (15772019, 0.5)]
User 15771961 neighbors : [(15771972, 1.0), (15772015, 0.94), (15771974, 0.93)]
User 15771934 neighbors : [(15771942, 1.0), (15772030, 0.98), (15771947, 0.96)]
User 15771998 neighbors : [(15772005, 0.89), (15771922, 0.88), (15771980, 0.66)]
User 15772036 neighbors : [(15771966, 0.97), (15771943, 0.94), (15771971, 0.62)]
User 15772012 neighbors : [(15772016, 0.94), (15771974, 0.58), (15771969, 0.58)]
User 15771948 neighbors : [(15771943, 0.94), (15771938, 0.73), (15771926, 0.59)]
User 15771976 neighbors : [(15771940, 0.85), (15771974, 0.58), (15771927, 0.53)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 1.0), (15771998, 0.66)]
User 15771940 neighbors : [(15771981, 0.98), (15771995, 0.97), (15771974, 0.94)]


In [18]:
def distance_euclidean(a,b):
    return 1 / (1+distance.euclidean(a,b))

In [19]:
for user in top_user['reviewNo']:
    print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(user, 3, distance_euclidean)))
    print()

User 15771936 neighbors : [(15771972, 0.25), (15771993, 0.2), (15772020, 0.12)]

User 15771961 neighbors : [(15772031, 0.33), (15771970, 0.33), (15771954, 0.33)]

User 15771934 neighbors : [(15772030, 0.41), (15771988, 0.29), (15772003, 0.25)]

User 15771998 neighbors : [(15771922, 0.17), (15772030, 0.16), (15772005, 0.15)]

User 15772036 neighbors : [(15771959, 1.0), (15771943, 0.24), (15772022, 0.19)]

User 15772012 neighbors : [(15772016, 0.22), (15772003, 0.15), (15771974, 0.13)]

User 15771948 neighbors : [(15771945, 0.41), (15771943, 0.23), (15771938, 0.15)]

User 15771976 neighbors : [(15771974, 0.31), (15771926, 0.31), (15771966, 0.29)]

User 15771977 neighbors : [(15771971, 1.0), (15771959, 0.5), (15771938, 0.25)]

User 15771940 neighbors : [(15771993, 1.0), (15771947, 1.0), (15771981, 0.33)]



# 2-b 영화 평점 예측

In [20]:
def nearest_neighbor_user_2(user, topN, simFunc):
    ul = UM_matrix_ds.loc[user].dropna()
    ratedIndex = ul.index
    nn= {}
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 =[]
        interSectionU2 =[]
        if uid == user: continue
        
        for i in ratedIndex:
            if not math.isnan(row[i]):
                
                interSectionU1.append(ul[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim): nn[uid] = sim
         
    return sorted(nn.items(), key = itemgetter(1))[:-(topN+1):-1]

In [39]:
def predict_rating(userid, nn=100, simFunc=distance_cosine):
    neighbor = nearest_neighbor_user_2(userid, nn, simFunc)
    neighbor_id = [id for id, sim in neighbor]
    
    neighbor_movie = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all', thresh=1)
    neighbor_dict = (dict(neighbor))
    ret =[]
    
    
    for movieId, row in neighbor_movie.iteritems():
        jsum, wsum =0 ,0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0],0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([movieId, wsum/jsum])
    
    return ret

result =[]
for i in range(10):
    userId = int(top_user.iloc[i].reviewNo)
    movieId = int(top_user.iloc[i].movieId)
    predict = predict_rating(userId, 300, distance_cosine)
    
    
    for movie in predict:
        if movieId == movie[0]:
            result.append([int(userId),int(movieId),movie[1]])
            
            
resultdf = pd.DataFrame(result, columns=['userId','movieId','rating'])
resultdf

{15771972: 0.9984038297885895, 15771993: 0.9715365547505717, 15772003: 0.946470152813405, 15771933: 0.9401021481358627, 15772020: 0.927594029147642, 15771959: 0.923558066153738, 15771945: 0.8973124579078625, 15771982: 0.8879351579115335, 15771943: 0.8868964797842298, 15772022: 0.874148756858934, 15772036: 0.8729482755183582, 15771938: 0.8710496683402693, 15771934: 0.8643105814329354, 15771961: 0.8638046503974647, 15771940: 0.8616404368553291, 15771925: 0.8584025892085594, 15771937: 0.8525684817280538, 15771977: 0.8515386887811751, 15772019: 0.8460475346763462, 15771998: 0.8429372176779683, 15771976: 0.8401508217737259, 15771969: 0.8283026881691261, 15772000: 0.8166063177175404, 15771932: 0.8122800538698337, 15771983: 0.8030573896702675, 15771948: 0.7938381724607979, 15771926: 0.7716978999558942, 15771971: 0.7456143892396931, 15771922: 0.7449285849342498, 15772012: 0.705508852002277, 15771995: 0.7041264395035616, 15771927: 0.6939864002577035, 15772016: 0.5915288163453278}
{15771947: 0.9

{15771971: 1.0, 15771959: 0.9984776046952725, 15771938: 0.99028229600855, 15771983: 0.9898334429608106, 15771961: 0.9534424167795305, 15771976: 0.9504559564113876, 15771934: 0.9452873632006791, 15771998: 0.9352298520149908, 15771982: 0.9326275721571906, 15772036: 0.9275521242559457, 15771926: 0.902227303436039, 15771948: 0.8708276796303962, 15771932: 0.8586483089620629, 15771936: 0.8515386887811751, 15772022: 0.8423515913404298, 15771927: 0.8164584804209641, 15771925: 0.8147609409799307, 15772012: 0.7087606927469234}
{15771993: 1.0, 15771947: 1.0, 15771981: 0.9958785014665825, 15771943: 0.9948057871850502, 15771976: 0.9939304498390475, 15772015: 0.9878783399072131, 15771926: 0.9783332144621877, 15771937: 0.9687464826690693, 15771961: 0.9684137284480767, 15771995: 0.9283786260935589, 15772006: 0.9178967532774625, 15771974: 0.9158576203334728, 15771934: 0.9082449928086991, 15772036: 0.9053050167489178, 15771933: 0.8934051474415642, 15771948: 0.8840571676144443, 15772022: 0.88113422106280

userId  movieId    rating
0  15771936   145162  5.796201
1  15771961   161967  8.595640
2  15771934   163788  9.150525
3  15771998   180399  7.000000
4  15772036    86507  8.561077
5  15771976   180399  7.000000
6  15771977   157297  5.800920

In [22]:
realdata_rating =[]
for userid in resultdf['userId']:
    realdata_rating.append(float(top_user[top_user['reviewNo']==userid]['rating']))
    
resultdata_rating = resultdf.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating,resultdata_rating)
print("\nError Rate(Avsolute) :",error_rate_absol)
print("\nError Rate(Squared) :",error_rate_squared)


Error Rate(Avsolute) : 1.798704122760153

Error Rate(Squared) : 5.6478653086387975


In [23]:
result =[]
for i in range(10):
    userId = int(top_user.iloc[i].reviewNo)
    movieId = int(top_user.iloc[i].movieId)
    predict = predict_rating(userId, 300, distance_correlation)
    
    for movie in predict:
        if movieId == movie[0]:
            result.append([int(userId),int(movieId),movie[1]])
            
resultdf = pd.DataFrame(result, columns=['userId','movieId','rating'])
resultdf

C:\Users\rrsg  aa   n odd12\Anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


[]
[]
[]
[]
[]
[]
[]
[]


C:\Users\rrsg  aa   n odd12\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


[]
[]


userId  movieId     rating
0  15771936   145162   5.433217
1  15771961   161967  10.715807
2  15771934   163788  15.406481
3  15771998   180399   7.000000
4  15772036    86507  11.304849
5  15771976   180399   7.000000
6  15771977   157297   0.954683

In [24]:
realdata_rating =[]
for userid in resultdf['userId']:
    realdata_rating.append(float(top_user[top_user['reviewNo']==userid]['rating']))
    
resultdata_rating = resultdf.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating,resultdata_rating)
print("\nError Rate(Avsolute) :",error_rate_absol)
print("\nError Rate(Squared) :",error_rate_squared)


Error Rate(Avsolute) : 1.8436674517673026

Error Rate(Squared) : 11.258077133072339


In [25]:

result =[]
for i in range(10):
    userId = int(top_user.iloc[i].reviewNo)
    movieId = int(top_user.iloc[i].movieId)
    predict = predict_rating(userId, 300, distance_euclidean)
    
    for movie in predict:
        if movieId == movie[0]:
            result.append([int(userId),int(movieId),movie[1]])
            
resultdf = pd.DataFrame(result, columns=['userId','movieId','rating'])
resultdf

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


userId  movieId    rating
0  15771936   145162  4.649008
1  15771961   161967  9.263031
2  15771934   163788  8.956179
3  15771998   180399  7.000000
4  15772036    86507  8.054289
5  15771976   180399  7.000000
6  15771977   157297  6.588583

In [26]:
realdata_rating =[]
for userid in resultdf['userId']:
    realdata_rating.append(float(top_user[top_user['reviewNo']==userid]['rating']))
    
resultdata_rating = resultdf.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating,resultdata_rating)
print("\nError Rate(Avsolute) :",error_rate_absol)
print("\nError Rate(Squared) :",error_rate_squared)


Error Rate(Avsolute) : 1.6966357708958737

Error Rate(Squared) : 6.0152917257030625
